In [ ]:
!pip install selenium
!pip install bs4
!pip requests
!pip install pandas
!pip install tqdm

ERROR: unknown command "requests"


In [ ]:
from os import environ
from selenium.webdriver import Remote
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chromium.remote_connection import ChromiumRemoteConnection as Connection
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import requests
import re
import warnings

warnings.filterwarnings("ignore")

def extract_hyperlinks_from_soup(soup, scraping_link):
    url_pattern = r'(https?://[^\s\\>"\'\)]+)'
    a_tags = soup.find_all('a')
    img_tags = soup.find_all('img')
    urls = re.findall(url_pattern, soup.text)
    hyperlinks = [a.get('href') for a in a_tags if a.get('href') and re.match(url_pattern, a.get('href'))]
    hyperlinks += [scraping_link[:-1] + str(a.get('href')) for a in a_tags if a.get('href') not in hyperlinks]
    hyperlinks.extend([img.get('src') for img in img_tags if img.get('src') and re.match(url_pattern, img.get('src'))])
    hyperlinks.extend(urls)

    cleaned_hyperlinks = []
    for url in hyperlinks:
        if url:
            url = url.split()[0]
            url = url.split('>')[0]
            cleaned_hyperlinks.append(url)

    return cleaned_hyperlinks

def find_about_us_link(links, soup, scraping_link):
    about_us_patterns = [
    r'about[-_ ]?us|about',
    r'our[-_ ]?story',
    r'who[-_ ]?we[-_ ]?are|who[-_ ]?are[-_ ]?we',
    r'about[-_ ]?our[-_ ]?company|our[-_ ]?company',
    r'our[-_ ]?mission',
    r'our[-_ ]?background',
    r'meet[-_ ]?the[-_ ]?team',
    r'what[-_ ]?we[-_ ]?do',
    r'company[-_ ]?overview',
    r'our[-_ ]?vision',
    r'our[-_ ]?values',
    r'corporate[-_ ]?info|corporate[-_ ]?information|corporate[-_ ]?details',
    r'company[-_ ]?info|company[-_ ]?information|company[-_ ]?details',
    r'get[-_ ]?to[-_ ]?know[-_ ]?us',
    r'learn[-_ ]?more[-_ ]?about[-_ ]?us',
    r'our[-_ ]?history',
    r'the[-_ ]?team',
    r'who[-_ ]?we[-_ ]?serve',
    r'why[-_ ]?choose[-_ ]?us',
    r'who[-_ ]?we[-_ ]?stand[-_ ]?for',
    r'our[-_ ]?philosophy',
    r'about[-_ ]?our[-_ ]?business',
    r'our[-_ ]?journey'
]

    a_tags = soup.find_all('a')
    about_us_links = []

    for a in a_tags:
        spans = a.find_all('span')

        for s in spans:
            for about_us in about_us_patterns:
                if re.search(about_us, s.text, re.IGNORECASE):
                    if scraping_link in str(a.get('href')):
                        about_us_links.append(str(a.get('href')))
                    else:
                        about_us_links.append(scraping_link + str(a.get('href')))
                    break

    for about_us in about_us_patterns:
        for link in links:
            if re.search(about_us, link, re.IGNORECASE):
                about_us_links.append(link)

    return about_us_links

def extract_relevant_text_from_about_us_page(soup):
    for tag in soup(['script', 'style', 'nav', 'footer', 'header', 'aside']):
        tag.decompose()

    paragraphs = soup.find_all('p')

    cleaned_paragraphs = []

    for p in paragraphs:
        text = p.get_text(strip=True)

        if len(text.split()) > 5:
            cleaned_paragraphs.append(text)

    relevant_text = "\n\n".join(cleaned_paragraphs)
    return relevant_text

if __name__ == '__main__':
    scraping_companies_websites = pd.read_csv('/content/scraping_companies_with_websites.csv')
    new_df_dict = {'name': [], 'website': [], 'scraped_text': []}

    for i in tqdm(range(len(scraping_companies_websites)), desc='Scraping...'):
        name, scraping_link = scraping_companies_websites.iloc[i]['name'], scraping_companies_websites.iloc[i]['website']
        try:
            response = requests.get(scraping_link, verify=False)
        except:
            continue

        scraped_text = ''

        if response.status_code == 200:

            soup = BeautifulSoup(response.text, 'html.parser')
            hyperlinks = extract_hyperlinks_from_soup(soup, scraping_link)

            about_us_links = find_about_us_link(hyperlinks, soup, scraping_link)
            visited_about_us_links = set()

            if about_us_links:
                for about_us_link in about_us_links:
                    if about_us_link not in visited_about_us_links:
                        try:
                            about_us_data = requests.get(about_us_link, verify=False)
                        except:
                            continue
                        if response.status_code == 200:
                            about_us_soup = BeautifulSoup(about_us_data.text, 'html.parser')
                            relevant_text = extract_relevant_text_from_about_us_page(about_us_soup)
                            scraped_text += relevant_text + '\n'

                            visited_about_us_links.add(about_us_link)

        if scraped_text == '':
            scraped_text = None

        new_df_dict['name'].append(name)
        new_df_dict['website'].append(scraping_link)
        new_df_dict['scraped_text'].append(scraped_text)

    scraped_df = pd.DataFrame(new_df_dict)
    scraped_df.to_csv('/content/scraping.csv', index=False)

Scraping...: 100%|██████████| 835/835 [1:25:11<00:00,  6.12s/it]


Error: need to escape, but no escapechar set

In [ ]:
df = pd.read_csv('/content/scraping.csv')
df = df.dropna().reset_index()
df = df[['name', 'website', 'scraped_text']]
df.to_csv('/content/scraping_without_nan.csv')
df

,name,website,scraped_text
0,Hand & Stone Massage and Facial Spa,http://handandstoneroundrock.com/,Hand & Stone Massage and Facial Spa is proud t...
1,Alpha Kappa Psi,https://akpsi.org/,"Trust, respect, cooperation,companionship and ..."
2,State of Texas,http://texas.gov/,We can't find the page you're looking for.Texa...
3,Franklin High School,http://frhs.egusd.net/,Elk Grove Unified School District is in the pr...
4,Ro Health,http://www.rohealth.com/,Ro Health is a rapidly growing medical staffin...
...,...,...,...
91,CalOptima,http://www.caloptima.org/,"In 2023, CalOptima Health launched the Compreh..."
92,"California State University, Monterey Bay",http://csumb.edu/,Privacy Policy Statement. Our Privacy Policy w...
93,California State University-San Bernardino,http://csusb.edu/,Mike Singer | CSUSB Office of Marketing and Co...
94,Calvin University,https://calvin.edu/,Calvin is a school with a big mission: equippi...
